# Variables, Types, and Functions

- Juliaにおけるsubtypeを理解する。
- branchingを扱うための多重ディスパッチの使い方。
- 変数としての関数の使い方。
- Juliaにおける関数型プログラミング。
- 変数のスコープ。
- 例外処理の扱い方。
- Named　Tuples　の働き。

----
## Understanding subtyping in Julia


----
## Usin multiple dispatch to handle branching behavoir
　多重ディスパッチを使えば，関数に与えられた引数の型に応じて，適切なメソッドが選択される。メソッドを定義する際は，引数に対して明示的に型を与える必要がある。

In [13]:
df = DataFrame(s = categorical(["a", "b", "c"]), 
               n = 1.0:3.0, 
               f = [sin, cos, missing])

# Defime multiple dispatch function
simpledescribe(v) = "unknown type"
simpledescribe(v::Vector{<:Number}) = "Numeric"
simpledescribe(v::CategoricalArray) = "catagorical"
simpledisplay(df) = foreach(x -> println(x[1], ": ", simpledescribe(x[2])), eachcol(df, true))

simpledisplay(df)

s: catagorical
n: Numeric
f: unknown type


次の例だと，fun1よりもfun2のほうが高速である。その理由はfun2ではコンパイルの間に，helper関数にデータの型が既知の状態で渡されるからである。

In [8]:
using DataFrames, BenchmarkTools
function helper(x)
    s = zero(eltype(x))
    for v in x
        s += v
    end
    s
end

function fun1(df)
        s = zero(eltype(df[!, 1]))
    for v in df[!, 1]
        s += v
    end
    s
end

fun2(df) = helper(df[!, 1])

fun2 (generic function with 1 method)

In [9]:
df = DataFrame(x = 1:10^6)
@btime fun1(df);

  58.617 ms (3998948 allocations: 76.28 MiB)


In [10]:
@btime fun2(df);

  123.053 μs (1 allocation: 16 bytes)


----
## Using functions as variables in Julia

----
## Handling exceptions in Julia

In [15]:
function loglines(filename, lines...)
    f = open(filename, "a")
    foreach(line -> (println(f, line)), lines)
    sqrt(-2)
    close(f)
end;

In [19]:
cd("/Users/takuizum/local_Documents/julia/Julia 1.0 Programming Cookbook")
pwd()

"/Users/takuizum/local_Documents/julia/Julia 1.0 Programming Cookbook"

In [22]:
loglines("mylog.txt", "Test.log:")

DomainError: DomainError with -2.0:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [23]:
try
    loglines("mylog.txt", "Test log:")
catch e
    dump(e)
end 

DomainError
  val: Float64 -2.0
  msg: String "sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x))."
